In [1]:
#Step -1
#importing the libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


# Set to display all the columns of the dataset
pd.set_option("display.max_columns",None)

#Ignore harmless warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
#Step -2
#Load the data
cdata=pd.read_csv(r"C:\Users\Praveen\OneDrive\Desktop\ABCD\newdata after EDA.csv",header=0)
#Copy to back_up file
cdata_bk=cdata.copy()
#Display first 5 records
cdata.head()

,age(in years),gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio,bmi,bmi_category,pulse_pressure,map
0,50.391781,2,168.0,62.0,110.0,80.0,1,1,0,0,1,0,21.967120,0,30.0,89.9
1,55.419178,1,156.0,85.0,140.0,90.0,3,1,0,0,1,1,34.927679,1,50.0,106.5
2,51.663014,1,165.0,64.0,130.0,70.0,3,1,0,0,0,1,23.507805,0,60.0,89.8
3,48.282192,2,169.0,82.0,150.0,100.0,1,1,0,0,1,1,28.710479,2,50.0,116.5
4,60.038356,1,151.0,67.0,120.0,80.0,2,2,0,0,0,0,29.384676,2,40.0,93.2


In [6]:
cdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62478 entries, 0 to 62477
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age(in years)   62478 non-null  float64
 1   gender          62478 non-null  int64  
 2   height          62478 non-null  float64
 3   weight          62478 non-null  float64
 4   ap_hi           62478 non-null  float64
 5   ap_lo           62478 non-null  float64
 6   cholesterol     62478 non-null  int64  
 7   gluc            62478 non-null  int64  
 8   smoke           62478 non-null  int64  
 9   alco            62478 non-null  int64  
 10  active          62478 non-null  int64  
 11  cardio          62478 non-null  int64  
 12  bmi             62478 non-null  float64
 13  bmi_category    62478 non-null  int64  
 14  pulse_pressure  62478 non-null  float64
 15  map             62478 non-null  float64
dtypes: float64(8), int64(8)
memory usage: 7.6 MB


In [7]:
pip install catboost

Note: you may need to restart the kernel to use updated packages.


In [10]:
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from catboost import CatBoostClassifier
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, balanced_accuracy_score, matthews_corrcoef, roc_auc_score
from imblearn.over_sampling import SMOTE

# Load your dataset
# Assuming 'data' is your dataset
# Define target column and features
target_column = 'cardio'
X = cdata.drop(columns=[target_column])
y = cdata[target_column]

# Define the CatBoost classifier
catboost = CatBoostClassifier(random_state=42)

# Define the hyperparameters grid
param_grid = {
    'iterations': [500, 1000, 1500, 2000],
    'learning_rate': [0.001, 0.01, 0.1, 0.2],
    'depth': [6, 8, 10, 12],
    'l2_leaf_reg': [1, 3, 5, 7],
    'border_count': [32, 64, 128],
    'subsample': [0.6, 0.7, 0.8, 0.9, 1.0],
    'colsample_bylevel': [0.6, 0.7, 0.8, 0.9, 1.0],
    'min_child_samples': [3, 5, 8, 10],
    'scale_pos_weight': [1, 2, 3],
    # ... Add more parameters as needed
}

# Different values for cross-validation
cv_values = [5, 10, 15, 20]
results = []  # Initialize the results list

for cv in cv_values:
    print(f"\nTrying with CV={cv}")

    # Split the dataset into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    # Perform RandomizedSearchCV for hyperparameter tuning
    random_search = RandomizedSearchCV(estimator=catboost, param_distributions=param_grid, n_iter=10, cv=cv, scoring='accuracy', verbose=1, n_jobs=-1)
    random_search.fit(X_train, y_train)

    # Get the best parameters and best score
    best_params = random_search.best_params_
    best_score = random_search.best_score_

    print("Best Parameters:", best_params)
    print("Best Score:", best_score)

    # Get the best model
    best_model = random_search.best_estimator_

    # Evaluate the best model on the test set
    y_pred = best_model.predict(X_test)

    # Calculate evaluation metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    balanced_accuracy = balanced_accuracy_score(y_test, y_pred)
    matthews_corr = matthews_corrcoef(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred)

    print("Evaluation Metrics:")
    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {f1}")
    print(f"Balanced Accuracy: {balanced_accuracy}")
    print(f"Matthews Corrcoef: {matthews_corr}")
    print(f"ROC AUC: {roc_auc}")

    # Store the results in a dictionary
    result_entry = {
        'CV': cv,
        'Best Parameters': best_params,
        'Best Score': best_score,
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1,
        'Balanced Accuracy': balanced_accuracy,
        'Matthews Corrcoef': matthews_corr,
        'ROC AUC': roc_auc
    }

    results.append(result_entry)

# Convert results to a DataFrame
results_df = pd.DataFrame(results)



Trying with CV=5
Fitting 5 folds for each of 10 candidates, totalling 50 fits
0:	learn: 0.6653506	total: 25.5ms	remaining: 50.9s
1:	learn: 0.6429199	total: 53ms	remaining: 52.9s
2:	learn: 0.6251000	total: 77.5ms	remaining: 51.6s
3:	learn: 0.6104363	total: 102ms	remaining: 50.8s
4:	learn: 0.5991793	total: 129ms	remaining: 51.3s
5:	learn: 0.5897403	total: 156ms	remaining: 51.9s
6:	learn: 0.5824159	total: 179ms	remaining: 50.9s
7:	learn: 0.5770105	total: 200ms	remaining: 49.9s
8:	learn: 0.5713253	total: 221ms	remaining: 48.8s
9:	learn: 0.5670326	total: 238ms	remaining: 47.3s
10:	learn: 0.5633900	total: 265ms	remaining: 47.9s
11:	learn: 0.5603645	total: 293ms	remaining: 48.6s
12:	learn: 0.5577163	total: 332ms	remaining: 50.7s
13:	learn: 0.5553199	total: 368ms	remaining: 52.2s
14:	learn: 0.5531998	total: 400ms	remaining: 53s
15:	learn: 0.5515589	total: 428ms	remaining: 53.1s
16:	learn: 0.5497450	total: 453ms	remaining: 52.8s
17:	learn: 0.5484807	total: 477ms	remaining: 52.6s
18:	learn: 0.5

162:	learn: 0.5076818	total: 3.55s	remaining: 40.1s
163:	learn: 0.5074919	total: 3.58s	remaining: 40s
164:	learn: 0.5073448	total: 3.6s	remaining: 40s
165:	learn: 0.5071235	total: 3.61s	remaining: 39.9s
166:	learn: 0.5068166	total: 3.63s	remaining: 39.8s
167:	learn: 0.5066543	total: 3.65s	remaining: 39.8s
168:	learn: 0.5065347	total: 3.66s	remaining: 39.7s
169:	learn: 0.5063807	total: 3.68s	remaining: 39.6s
170:	learn: 0.5062372	total: 3.7s	remaining: 39.6s
171:	learn: 0.5059640	total: 3.72s	remaining: 39.5s
172:	learn: 0.5057711	total: 3.73s	remaining: 39.4s
173:	learn: 0.5056338	total: 3.75s	remaining: 39.3s
174:	learn: 0.5055312	total: 3.76s	remaining: 39.3s
175:	learn: 0.5053514	total: 3.78s	remaining: 39.2s
176:	learn: 0.5051932	total: 3.8s	remaining: 39.2s
177:	learn: 0.5050129	total: 3.83s	remaining: 39.2s
178:	learn: 0.5048639	total: 3.85s	remaining: 39.1s
179:	learn: 0.5046011	total: 3.87s	remaining: 39.1s
180:	learn: 0.5043423	total: 3.88s	remaining: 39s
181:	learn: 0.5041454

329:	learn: 0.4795670	total: 6.78s	remaining: 34.3s
330:	learn: 0.4792821	total: 6.81s	remaining: 34.3s
331:	learn: 0.4790958	total: 6.83s	remaining: 34.3s
332:	learn: 0.4790202	total: 6.85s	remaining: 34.3s
333:	learn: 0.4789153	total: 6.87s	remaining: 34.3s
334:	learn: 0.4787412	total: 6.89s	remaining: 34.2s
335:	learn: 0.4785416	total: 6.91s	remaining: 34.2s
336:	learn: 0.4783983	total: 6.92s	remaining: 34.2s
337:	learn: 0.4783233	total: 6.94s	remaining: 34.1s
338:	learn: 0.4781980	total: 6.96s	remaining: 34.1s
339:	learn: 0.4780818	total: 6.98s	remaining: 34.1s
340:	learn: 0.4778722	total: 7s	remaining: 34s
341:	learn: 0.4776287	total: 7.02s	remaining: 34s
342:	learn: 0.4775429	total: 7.04s	remaining: 34s
343:	learn: 0.4774050	total: 7.05s	remaining: 34s
344:	learn: 0.4772626	total: 7.07s	remaining: 33.9s
345:	learn: 0.4771552	total: 7.09s	remaining: 33.9s
346:	learn: 0.4770286	total: 7.11s	remaining: 33.9s
347:	learn: 0.4768097	total: 7.14s	remaining: 33.9s
348:	learn: 0.4766289	t

495:	learn: 0.4558610	total: 10s	remaining: 30.4s
496:	learn: 0.4557633	total: 10.1s	remaining: 30.4s
497:	learn: 0.4556335	total: 10.1s	remaining: 30.4s
498:	learn: 0.4555629	total: 10.1s	remaining: 30.3s
499:	learn: 0.4554634	total: 10.1s	remaining: 30.3s
500:	learn: 0.4553412	total: 10.1s	remaining: 30.3s
501:	learn: 0.4552544	total: 10.2s	remaining: 30.3s
502:	learn: 0.4550374	total: 10.2s	remaining: 30.3s
503:	learn: 0.4548881	total: 10.2s	remaining: 30.3s
504:	learn: 0.4547440	total: 10.2s	remaining: 30.2s
505:	learn: 0.4546178	total: 10.2s	remaining: 30.2s
506:	learn: 0.4545092	total: 10.3s	remaining: 30.2s
507:	learn: 0.4543866	total: 10.3s	remaining: 30.2s
508:	learn: 0.4542071	total: 10.3s	remaining: 30.2s
509:	learn: 0.4539762	total: 10.3s	remaining: 30.2s
510:	learn: 0.4539018	total: 10.3s	remaining: 30.1s
511:	learn: 0.4536711	total: 10.4s	remaining: 30.1s
512:	learn: 0.4534768	total: 10.4s	remaining: 30.1s
513:	learn: 0.4534200	total: 10.4s	remaining: 30.1s
514:	learn: 0.

657:	learn: 0.4348557	total: 13s	remaining: 26.5s
658:	learn: 0.4347612	total: 13s	remaining: 26.5s
659:	learn: 0.4346127	total: 13.1s	remaining: 26.5s
660:	learn: 0.4344569	total: 13.1s	remaining: 26.5s
661:	learn: 0.4342897	total: 13.1s	remaining: 26.5s
662:	learn: 0.4341804	total: 13.1s	remaining: 26.4s
663:	learn: 0.4341000	total: 13.1s	remaining: 26.4s
664:	learn: 0.4340213	total: 13.2s	remaining: 26.4s
665:	learn: 0.4338994	total: 13.2s	remaining: 26.4s
666:	learn: 0.4337521	total: 13.2s	remaining: 26.4s
667:	learn: 0.4336768	total: 13.2s	remaining: 26.3s
668:	learn: 0.4335829	total: 13.2s	remaining: 26.3s
669:	learn: 0.4335139	total: 13.2s	remaining: 26.3s
670:	learn: 0.4334013	total: 13.2s	remaining: 26.2s
671:	learn: 0.4332702	total: 13.3s	remaining: 26.2s
672:	learn: 0.4330845	total: 13.3s	remaining: 26.2s
673:	learn: 0.4330219	total: 13.3s	remaining: 26.2s
674:	learn: 0.4328770	total: 13.3s	remaining: 26.1s
675:	learn: 0.4327485	total: 13.3s	remaining: 26.1s
676:	learn: 0.43

...

220:	learn: 0.5237857	total: 2.57s	remaining: 9.06s
221:	learn: 0.5237398	total: 2.59s	remaining: 9.07s
222:	learn: 0.5236355	total: 2.6s	remaining: 9.07s
223:	learn: 0.5235494	total: 2.62s	remaining: 9.06s
224:	learn: 0.5234384	total: 2.63s	remaining: 9.05s
225:	learn: 0.5233766	total: 2.64s	remaining: 9.04s
226:	learn: 0.5233017	total: 2.65s	remaining: 9.03s
227:	learn: 0.5232088	total: 2.66s	remaining: 9.02s
228:	learn: 0.5231390	total: 2.67s	remaining: 9.01s
229:	learn: 0.5230170	total: 2.69s	remaining: 9s
230:	learn: 0.5229435	total: 2.7s	remaining: 8.98s
231:	learn: 0.5228400	total: 2.71s	remaining: 8.97s
232:	learn: 0.5227411	total: 2.72s	remaining: 8.96s
233:	learn: 0.5226417	total: 2.73s	remaining: 8.95s
234:	learn: 0.5225881	total: 2.75s	remaining: 8.94s
235:	learn: 0.5225120	total: 2.76s	remaining: 8.93s
236:	learn: 0.5224169	total: 2.77s	remaining: 8.92s
237:	learn: 0.5223438	total: 2.79s	remaining: 8.92s
238:	learn: 0.5222071	total: 2.8s	remaining: 8.91s
239:	learn: 0.5220

379:	learn: 0.5103086	total: 4.68s	remaining: 7.64s
380:	learn: 0.5102858	total: 4.7s	remaining: 7.64s
381:	learn: 0.5101908	total: 4.72s	remaining: 7.63s
382:	learn: 0.5101658	total: 4.73s	remaining: 7.62s
383:	learn: 0.5100587	total: 4.75s	remaining: 7.61s
384:	learn: 0.5099506	total: 4.76s	remaining: 7.6s
385:	learn: 0.5098873	total: 4.77s	remaining: 7.59s
386:	learn: 0.5098110	total: 4.78s	remaining: 7.58s
387:	learn: 0.5097457	total: 4.79s	remaining: 7.56s
388:	learn: 0.5096714	total: 4.81s	remaining: 7.55s
389:	learn: 0.5096048	total: 4.82s	remaining: 7.54s
390:	learn: 0.5095265	total: 4.83s	remaining: 7.53s
391:	learn: 0.5094277	total: 4.85s	remaining: 7.52s
392:	learn: 0.5093090	total: 4.86s	remaining: 7.51s
393:	learn: 0.5092712	total: 4.87s	remaining: 7.5s
394:	learn: 0.5092101	total: 4.88s	remaining: 7.48s
395:	learn: 0.5091314	total: 4.9s	remaining: 7.48s
396:	learn: 0.5090472	total: 4.92s	remaining: 7.47s
397:	learn: 0.5089557	total: 4.93s	remaining: 7.46s
398:	learn: 0.50

546:	learn: 0.4975928	total: 6.98s	remaining: 5.78s
547:	learn: 0.4975008	total: 7s	remaining: 5.77s
548:	learn: 0.4973940	total: 7.01s	remaining: 5.76s
549:	learn: 0.4973238	total: 7.03s	remaining: 5.75s
550:	learn: 0.4972545	total: 7.04s	remaining: 5.74s
551:	learn: 0.4972122	total: 7.05s	remaining: 5.72s
552:	learn: 0.4971397	total: 7.07s	remaining: 5.71s
553:	learn: 0.4970613	total: 7.08s	remaining: 5.7s
554:	learn: 0.4969702	total: 7.09s	remaining: 5.68s
555:	learn: 0.4969074	total: 7.1s	remaining: 5.67s
556:	learn: 0.4968221	total: 7.12s	remaining: 5.66s
557:	learn: 0.4967663	total: 7.13s	remaining: 5.65s
558:	learn: 0.4966941	total: 7.14s	remaining: 5.63s
559:	learn: 0.4966203	total: 7.15s	remaining: 5.62s
560:	learn: 0.4965935	total: 7.17s	remaining: 5.61s
561:	learn: 0.4964925	total: 7.18s	remaining: 5.59s
562:	learn: 0.4964139	total: 7.19s	remaining: 5.58s
563:	learn: 0.4963335	total: 7.21s	remaining: 5.57s
564:	learn: 0.4962591	total: 7.22s	remaining: 5.56s
565:	learn: 0.496

717:	learn: 0.4869198	total: 9.29s	remaining: 3.65s
718:	learn: 0.4868398	total: 9.3s	remaining: 3.64s
719:	learn: 0.4867698	total: 9.32s	remaining: 3.62s
720:	learn: 0.4867177	total: 9.33s	remaining: 3.61s
721:	learn: 0.4866397	total: 9.34s	remaining: 3.6s
722:	learn: 0.4865735	total: 9.36s	remaining: 3.58s
723:	learn: 0.4865045	total: 9.37s	remaining: 3.57s
724:	learn: 0.4864538	total: 9.38s	remaining: 3.56s
725:	learn: 0.4864290	total: 9.39s	remaining: 3.54s
726:	learn: 0.4863577	total: 9.41s	remaining: 3.53s
727:	learn: 0.4862990	total: 9.42s	remaining: 3.52s
728:	learn: 0.4862433	total: 9.44s	remaining: 3.51s
729:	learn: 0.4861682	total: 9.45s	remaining: 3.49s
730:	learn: 0.4861242	total: 9.46s	remaining: 3.48s
731:	learn: 0.4860485	total: 9.47s	remaining: 3.47s
732:	learn: 0.4859909	total: 9.48s	remaining: 3.46s
733:	learn: 0.4859539	total: 9.5s	remaining: 3.44s
734:	learn: 0.4858740	total: 9.52s	remaining: 3.43s
735:	learn: 0.4858162	total: 9.53s	remaining: 3.42s
736:	learn: 0.4

877:	learn: 0.4776807	total: 11.4s	remaining: 1.59s
878:	learn: 0.4776409	total: 11.5s	remaining: 1.58s
879:	learn: 0.4776131	total: 11.5s	remaining: 1.56s
880:	learn: 0.4775695	total: 11.5s	remaining: 1.55s
881:	learn: 0.4775342	total: 11.5s	remaining: 1.54s
882:	learn: 0.4775142	total: 11.5s	remaining: 1.52s
883:	learn: 0.4774559	total: 11.5s	remaining: 1.51s
884:	learn: 0.4773948	total: 11.5s	remaining: 1.5s
885:	learn: 0.4773679	total: 11.5s	remaining: 1.48s
886:	learn: 0.4773332	total: 11.6s	remaining: 1.47s
887:	learn: 0.4773019	total: 11.6s	remaining: 1.46s
888:	learn: 0.4772502	total: 11.6s	remaining: 1.45s
889:	learn: 0.4771780	total: 11.6s	remaining: 1.43s
890:	learn: 0.4771153	total: 11.6s	remaining: 1.42s
891:	learn: 0.4770485	total: 11.6s	remaining: 1.41s
892:	learn: 0.4769853	total: 11.6s	remaining: 1.39s
893:	learn: 0.4769306	total: 11.6s	remaining: 1.38s
894:	learn: 0.4768507	total: 11.7s	remaining: 1.37s
895:	learn: 0.4767710	total: 11.7s	remaining: 1.35s
896:	learn: 0

In [11]:
results_df

,CV,Best Parameters,Best Score,Accuracy,Precision,Recall,F1 Score,Balanced Accuracy,Matthews Corrcoef,ROC AUC
0,5,"{'subsample': 0.6, 'scale_pos_weight': 1, 'min...",0.709196,0.710574,0.721356,0.662913,0.690901,0.709452,0.421029,0.709452
1,10,"{'subsample': 0.7, 'scale_pos_weight': 1, 'min...",0.729867,0.732768,0.750697,0.677236,0.712077,0.731460,0.466132,0.731460
2,15,"{'subsample': 0.9, 'scale_pos_weight': 1, 'min...",0.729067,0.732341,0.749607,0.677892,0.711948,0.731059,0.465205,0.731059
3,20,"{'subsample': 0.9, 'scale_pos_weight': 1, 'min...",0.723899,0.726419,0.741176,0.675049,0.706569,0.725210,0.453097,0.725210


In [12]:
# Save results to a CSV file
results_df.to_csv('catboost_hyperparameter_tuning_results.csv', index=False)